<p><font size="7" color='grey'> <b>
Anwendung Generativer KI
</b></font> </br></p>

<p><font size="6" color='grey'> <b>
LangChain 101
</b></font> </br></p>

---

In [ ]:
#@title
#@markdown   <p><font size="4" color='green'>  Colab-Umfeld</font> </br></p>
# Installierte Python Version
import sys
print(f"Python Version: ",sys.version)
# Installierte LangChain Bibliotheken
print()
print("Installierte LangChain Bibliotheken:")
!pip list | grep '^langchain'
# Unterdrückt die "DeprecationWarning" von LangChain für die Memory-Funktionden
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="langsmith.client")

In [ ]:
#@title
#@markdown   <p><font size="4" color='green'>  SetUp API-Keys (setup_api_keys)</font> </br></p>
def setup_api_keys():
    """Konfiguriert alle benötigten API-Keys aus Google Colab userdata"""
    from google.colab import userdata
    import os
    from os import environ

    # Dictionary der benötigten API-Keys
    keys = {
        'OPENAI_API_KEY': 'OPENAI_API_KEY',
        'HF_TOKEN': 'HF_TOKEN',
        # Weitere Keys bei Bedarf
    }

    # Keys in Umgebungsvariablen setzen
    for env_var, key_name in keys.items():
        environ[env_var] = userdata.get(key_name)

    return {k: environ[k] for k in keys.keys()}

# Verwendung
all_keys = setup_api_keys()
# Bei Bedarf einzelne Keys direkt zugreifen
# WEATHER_API_KEY = all_keys['WEATHER_API_KEY']

# **1 | Was ist LangChain?**
---


# 1 | Was ist LangChain?
---

LangChain ist ein Framework zur **Entwicklung von Anwendungen mit großen Sprachmodellen** (LLMs). Es vereinfacht die Integration von LLMs in eigene Anwendungen durch:

- Modulare Komponenten für verschiedene Aufgaben
- Vorgefertigte Ketten (Chains) für komplexe Workflows
- Einfache Integration externer Datenquellen
- Werkzeuge für das Speichern von Konversationskontexten



[Einführung](https://python.langchain.com/docs/introduction/)   
[Konzepte](https://python.langchain.com/docs/concepts/)   
[API-References](https://python.langchain.com/api_reference/index.html)   
[Integrations](https://python.langchain.com/docs/integrations/providers/)

---

[Tutorials](https://python.langchain.com/docs/tutorials/)   
[How-to-Guides](https://python.langchain.com/docs/how_to/)   

In [ ]:
!uv pip install --system -q langchain_community langchain_openai

In [ ]:
from os import environ
from google.colab import userdata
environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

# **2 | Kernkonzepte**
---




## 2.1 Message-Typen

Beim Einsatz von Large Language Models (LLMs) wie GPT gibt es drei Hauptarten von Nachrichten (Messages), die die Interaktion mit dem Modell steuern. Diese Messages definieren, wie das Modell reagiert, welche Eingaben es erhält und welche Ausgaben es generiert.



**System Message:**   
Die `System Message` dient dazu, das Verhalten des LLMs zu steuern. Sie ist eine nicht sichtbare Nachricht, die dem Modell Anweisungen gibt, wie es sich verhalten soll. Dabei kann sie die Persönlichkeit, den Tonfall und die Einschränkungen des Modells beeinflussen. Die Systemnachricht funktioniert wie eine Art "Rollenspiel-Anweisung", um das Modell in eine gewünschte Rolle zu versetzen.

*Merkmale:*
- Definiert, wie das Modell reagieren soll
- Legt Persönlichkeit, Verhalten und Einschränkungen fest
- Wird zu Beginn einer Sitzung gesetzt und bleibt bestehen
- Nicht sichtbar für den Benutzer

*Beispiel einer System Message:*
```json
{
  "role": "system",
  "content": "You are an AI assistant that provides concise and informative answers in a professional tone."
}
```
Dieses Beispiel weist das Modell an, in einem professionellen Tonfall kurze und informative Antworten zu geben.

---



**User Message**   
Die `User Message` ist die eigentliche Eingabe des Benutzers. Sie kann eine Frage, eine Aufforderung oder ein Befehl sein. Das Modell nutzt diese Nachricht als Ausgangspunkt für die Generierung einer Antwort.

*Merkmale:*
- Direkte Eingabe des Benutzers
- Kann eine Frage, Anfrage oder einen Befehl enthalten
- Basis für die Antwort des Modells

*Beispiel einer User Message:*
```json
{
  "role": "user",
  "content": "What are the key differences between AI and machine learning?"
}
```
Hier fragt der Benutzer nach den Unterschieden zwischen KI und maschinellem Lernen.

---


**AI Message**   
Die `AI Message` ist die Antwort, die das Modell generiert. Sie basiert auf der User Message und den Anweisungen aus der System Message. Die AI Message kann verschiedene Eigenschaften haben, ist aber hauptsächlich auf den Inhalt fokussiert.

*Merkmale:*
- Antwort des Modells auf die Benutzeranfrage
- Kann verschiedene Eigenschaften haben (z. B. Länge, Stil)
- Hauptsächlich inhaltlich relevant

*Beispiel einer AI Message:*
```json
{
  "role": "assistant",
  "content": "Artificial Intelligence (AI) is a broad field that includes various subfields, one of which is Machine Learning (ML). ML focuses on creating models that learn from data and make predictions."
}
```
Hier gibt das Modell eine inhaltliche Antwort auf die Frage des Benutzers.

---



**Zusammenhang**   
Die drei Message-Typen arbeiten zusammen, um eine sinnvolle Interaktion mit dem Modell zu ermöglichen:

+  **System Message** legt die Regeln und das Verhalten des Modells fest.
+ **User Message** stellt eine Anfrage oder einen Befehl.
+ **AI Message** gibt die generierte Antwort basierend auf den vorherigen Nachrichten.

**Beispiel für eine vollständige Konversation:**    
```json
[
  {
    "role": "system",
    "content": "You are a helpful AI assistant that provides educational answers in a friendly tone."
  },
  {
    "role": "user",
    "content": "Can you explain what Large Language Models are?"
  },
  {
    "role": "assistant",
    "content": "Sure! Large Language Models (LLMs) are advanced AI systems trained on vast amounts of text data. They can understand and generate human-like text, making them useful for various applications like chatbots, content creation, and more."
  }
]
```
In diesem Beispiel wurde das Modell so eingestellt, dass es in einem freundlichen Tonfall antwortet, dann erhält es eine Benutzeranfrage und gibt eine passende Antwort.

---



In [ ]:
!uv pip install --system -q langchain_openai

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_openai import ChatOpenAI
from IPython.display import display, Markdown

MODEL = 'gpt-4o-mini'
TEMPERATURE = 0.0

messages = [
    SystemMessage(
        content="Du bist ein kompetenter und hilfreicher Assistent."
    ),
    HumanMessage(
        content="Nenne mir die Hauptstadt von Usbekistan?"
    ),
]


# Initialisieren Sie das OpenAI LLM mit Ihrem API-Key
llm = ChatOpenAI(
  model=MODEL,
  temperature= TEMPERATURE,
)

display(Markdown("## 📣 Model response:"))
display(Markdown("---"))
output = llm.invoke(messages)
display(Markdown(output.content))

In [ ]:
messages.append(output)
messages.append(HumanMessage(content="Bis Du sicher? Ich glaube, sie wurde kürzlich umbenannt."))

for message in messages:
    print(message)

Wir können das Konversationsarray an das Modell übermitteln und seine neueste Antwort anzeigen.

In [ ]:
display(Markdown("## 📣 Model response:"))
display(Markdown("---"))
output = llm.invoke(messages)
display(Markdown(output.content))

**Fazit**    
Die System-, User- und AI-Message sind essenzielle Bestandteile jeder Interaktion mit einem LLM. Während die System Message das Verhalten vorgibt, stellt die User Message die eigentliche Anfrage dar, auf die das Modell mit einer AI Message antwortet. Dieses Konzept ist zentral für den Einsatz von LLMs in Chatbots, KI-gestützten Assistenten und anderen interaktiven Anwendungen.



## 2.2 Prompts

Prompts sind die Eingaben bzw. Aufträge an das LLM. LangChain bietet verschiedene Template-Systeme:

<p><font color='black' size="5">
Simple Prompt
</font></p>

In [ ]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate

# 1. Einfaches Template mit Platzhaltern
template = PromptTemplate(
    input_variables=["thema"],
    template="Erkläre {thema} in einfachen Worten."
)

In [ ]:
print(template.format(thema="Machine Learning"))


<p><font color='black' size="5">
Eingabeaufforderungsvorlagen
</font></p>



Mit LangChain können Sie Operationsketten erstellen, die normalerweise als Teil einer LLM-fähigen Anwendung ausgeführt werden. Eine dieser Operationen ist eine Eingabeaufforderungsvorlage, mit der Sie Text in eine zuvor erstellte Eingabeaufforderung einfügen können. In diesem Beispiel erstellen wir eine Eingabeaufforderungsvorlage, die das Modell auffordert, einen zufälligen Titel für einen Blogbeitrag zu erstellen.

```
Gibt nur den Titel eines Blog-Beitragsartikels zum Thema {topic} in {language} zurück.
```

Um dieses Ziel zu erreichen, verwenden wir ein **PromptTemplate**-Objekt.

In [ ]:
from langchain.prompts import PromptTemplate

topic = "Haustiere für Datenwissenschaftler"
language = "german"

# Höhere Temperaturen für mehr Kreativität
llm = ChatOpenAI(model=MODEL, temperature=0.7)

# Definieren der Eingabeaufforderungsvorlage
title_template = PromptTemplate(
    input_variables=['topic', 'language'],
    template='Gibt nur den Titel eines Blog-Beitragsartikels zum Thema {topic} in {language} zurück.'
)

# Verkettung
title_chain = title_template | llm

# Aufrufen der Kette mit Eingaben
response = title_chain.invoke({'topic': topic, 'language': language})
print(response.content)

<p><font color='black' size="5">
Chat Prompt
</font></p>

In [ ]:
chat_template = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein hilfreicher Assistent."),
    ("human", "Erkläre mir {thema}"),
    ("assistant", "Ich erkläre dir {thema} gerne.")
])

In [ ]:
print(chat_template.format(thema="Machine Learning"))

<p><font color='black' size="5">
Few-Shot-Learning Prompt
</font></p>

In [ ]:
examples = [
    {"frage": "Was ist Python?", "antwort": "Eine Programmiersprache."},
    {"frage": "Was ist Java?", "antwort": "Eine objektorientierte Sprache."}
]

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=PromptTemplate(
        input_variables=["frage", "antwort"],
        template="Frage: {frage}\nAntwort: {antwort}"
    ),
    prefix="Beantworte die Frage basierend auf den Beispielen:",
    suffix="Frage: {input}\nAntwort:",
    input_variables=["input"]
)

In [ ]:
print(few_shot_prompt.format(input="Was ist Cobol?"))

## 2.3 Modelle

LangChain unterstützt verschiedene LLMs und bietet flexible Konfigurationsmöglichkeiten:

In [ ]:
from langchain_community.chat_models import ChatOpenAI
from langchain_openai import OpenAI

# Text-Completion-Modell mit Parametern
llm = OpenAI(
    model_name="gpt-3.5-turbo-instruct",
    temperature=0.7,
    max_tokens=500
)

# Chat-Modell für Konversationen
chat_model = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.9
)

## 2.4 LCEL aka Chains

Chains verbinden mehrere Komponenten zu einer Verarbeitungskette:

<p><font color='black' size="5">
LangChain Expression Language (LCEL)
</font></p>

LangChain Expression Language (LCEL) ist das **neue Programmiermodell in LangChain v0.3+**, das die Entwicklung von LLM-Anwendungen vereinfacht.

Hier die Kernpunkte:

1. Was ist LCEL?
```python
# LCEL nutzt den Pipe-Operator (|) für klare Verkettungen
chain = prompt | model | output_parser
```
- Eine deklarative Sprache zum Verketten von LangChain-Komponenten
- Ermöglicht linearen Datenfluss zwischen Komponenten
- Basiert auf dem Pipe-Operator (|) für intuitive Verbindungen

2. Warum LCEL nutzen?
- Bessere Lesbarkeit des Codes
- Einfachere Wartung und Debugging
- Verbesserte Performance durch optimierte Ausführung
- Bessere Typsicherheit und Fehlererkennung
- Unterstützt modernes Streaming und Async-Operationen

3. Praktisches Beispiel:
```python
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
# Einfache LCEL-Kette
chain = (
    ChatPromptTemplate.from_messages([("human", "{input}")])
    | ChatOpenAI()
    | StrOutputParser()
)
# Ausführung
result = chain.invoke({"input": "Erkläre LCEL"})
```

LCEL ist der empfohlene Weg für alle neuen LangChain-Projekte, da es die Entwicklung vereinfacht und zukunftssicher macht.

Für komplexere Anwendungen mit umfangreichem Zustandsmanagement, Verzweigungen oder mehreren Agenten wird die Verwendung von LangGraph empfohlen

In [ ]:
from langchain_community.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Konstanten
MODEL="gpt-4o-mini",
TEMPERATUR=0.7

# Modell-Konfiguration
chat_model = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.7
)

<p><font color='black' size="5">
Simple Chain
</font></p>

In [ ]:
# Prompt
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein hilfreicher Assistent."),
    ("human", "{eingabe}")
])

In [ ]:
# Verarbeitungskette
simple_chain = (
    prompt_template
    | chat_model
    | StrOutputParser()
)

In [ ]:
# Aufruf
result = simple_chain.invoke({"eingabe": "Was ist Machine Learning?"})
print("Simple Chain Ergebnis:", result)

<p><font color='black' size="5">
Sequential Chains
</font></p>

Sequentielle Chain: Linear, ein Input wird schrittweise verarbeitet

In [ ]:
# Prompts
zusammenfassen_template = ChatPromptTemplate.from_messages([
    ("system", "Fasse den folgenden Text prägnant zusammen."),
    ("human", "{text}")
])

uebersetzen_template = ChatPromptTemplate.from_messages([
    ("system", "Übersetze den folgenden Text ins Deutsche."),
    ("human", "{text}")
])

In [ ]:
# Verarbeitungskette
sequential_chain = (
    zusammenfassen_template
    | chat_model
    | StrOutputParser()
    | (lambda x: {"text": x})  # Ausgabe der ersten Chain als Eingabe für die zweite
    | uebersetzen_template
    | chat_model
    | StrOutputParser()
)

In [ ]:
# Aufruf
text = """
Machine Learning is a subset of artificial intelligence that focuses on developing
systems that can learn and improve from experience without being explicitly programmed.
"""
seq_result = sequential_chain.invoke({"text": text})
print("\nSequential Chain Ergebnis:", seq_result)

<p><font color='black' size="5">
Q&A-Chain
</font></p>

QA Chain: Kontextbasiert, kombiniert mehrere Inputs für eine einzelne Verarbeitung

In [ ]:
# Prompt
qa_template = ChatPromptTemplate.from_messages([
    ("system", "Beantworte die Frage basierend auf dem gegebenen Kontext."),
    ("human", """
    Kontext: {context}

    Frage: {question}
    """)
])

In [ ]:
# Verkettung
qa_chain = (
    qa_template
    | chat_model
    | StrOutputParser()
)

In [ ]:
# Aufruf
context = """
Python ist eine beliebte Programmiersprache für Machine Learning.
Sie bietet viele Bibliotheken wie TensorFlow und PyTorch.
"""
question = "Welche ML-Bibliotheken gibt es für Python?"
qa_result = qa_chain.invoke({
    "context": context,
    "question": question
})
print("\nQA Chain Ergebnis:", qa_result)

In [ ]:
# Formatierte Ausgabe
from IPython.display import display, Markdown
display(Markdown(qa_result))

<p><font color='black' size="5">
invoke - batch - stream
</font></p>

Aufruf der Verarbeitungskette erfolgt über:   
+ invoke(): Für einzelne, einfache Anfragen
+ batch(): Wenn mehrere Anfragen parallel verarbeitet werden sollen
+ stream(): Für lange Antworten oder bessere Nutzererfahrung

In [ ]:
template = PromptTemplate(
    input_variables=["thema"],
    template="Erkläre {thema} in einfachen Worten."
)
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein hilfreicher Assistent."),
    ("human", "{eingabe}")
])
output_parser = StrOutputParser()

# LCEL-Verkettung mit Pipe-Operator
chain = prompt_template | chat_model | output_parser

In [ ]:
# ---invoke--- #
result_with_invoke = chain.invoke("hello world!")
print(result_with_invoke)

In [ ]:
# ---batch--- #
result_with_batch = chain.batch(["hello", "world", "!"])
print(result_with_batch)

In [ ]:
# ---stream--- #
for chunk in chain.stream("Karl ist 22 Jahre alt."):
    print(chunk, flush=True)

<p><font color='black' size="5">
Vergleich Sequential vs Q&A
</font></p>

| Aspekt | Sequentielle Chain | QA Chain |
|--------|-------------------|-----------|
| Eingabe | Einzelner Text | Kontext + Frage |
| Verarbeitung | Schrittweise | Parallel |
| Ausgabe | Transformierter Text | Kontextbezogene Antwort |
| Anwendung | Textverarbeitung | Informationsextraktion |
| Flexibilität | Fest definiert | Kontextabhängig |

Verschoben aus M01 - redundant?


<p><font color='black' size="5">
Erstellen einer einfache sequentielle Kette
</font></p>



Wir werden nun LangChain verwenden, um mehrere LLM-Aufrufe mithilfe der Klasse **SimpleSequentialChain** zu einer längeren Kette zusammenzufassen. Wir werden zwei kleinere Ketten verwenden, um einen Titel und einen Textkörper für einen Blogbeitrag zu erstellen. Wir beginnen mit der Definition der beiden Eingabeaufforderungen, die wir zum Erstellen dieses Blogbeitrags verwenden werden. Beachten Sie auch, dass wir verlangen, dass das LLM [markdown](https://en.wikipedia.org/wiki/Markdown) verwendet, um den eigentlichen Blogbeitrag zu generieren.


In [ ]:
# Erstellen Sie die beiden Eingabeaufforderungsvorlagen
title_template = PromptTemplate( input_variables = ['topic'], template = 'Gibt nur den Titel eines Blog-Beitragsartikels zu {topic} in Deutsch' )
article_template = PromptTemplate( input_variables = ['title'], template = 'Schreibe einen Blog-Beitrag {title}, das Format ist Markdown.' )

Wir erstellen die erste Kette, um den zufälligen Titel zu generieren. Hier können die Benutzer das Thema angeben. Wir verwenden eine höhere Temperatur, um die Kreativität des Titels zu steigern. Wir verwenden auch ein einfacheres Modell, um die Kosten für die relativ einfache Aufgabe der Titelauswahl zu minimieren.

In [ ]:
MODEL = 'gpt-4o-mini'

# Erstellen Sie eine Kette, um einen Zufallsgenerator zu erzeugen.
llm = ChatOpenAI(model=MODEL, temperature=0.7)
title_chain = title_template | llm

Als nächstes verfassen wir den eigentlichen Blogbeitrag. Wir verwenden eine niedrigere Temperatur, um die Kreativität zu verringern und das LLM dazu zu bringen, sich an sachliche Informationen zu halten und Halluzinationen zu vermeiden. Wir verwenden auch ein komplexeres Modell, um einen besseren Artikel zu erstellen.

In [ ]:
MODEL2 = 'gpt-4o'

# Erstellen der Artikelkette
llm2 = ChatOpenAI(model=MODEL2, temperature=0.1)
article_chain = article_template | llm2

Nun kombinieren wir diese beiden Ketten zu einer. Die Eingabe für die erste Kette ist das ausgewählte Thema. Die erste Kette gibt dann den Titel an die zweite Kette aus, die wiederum den eigentlichen Artikel ausgibt.

In [ ]:
# Erstellen einer vollständige Kette, um einen neuen Blogbeitrag zu erstellen
complete_chain = title_chain | article_chain

Wir können nun den fertigen Artikel anzeigen. In diesem Fall haben wir einen Artikel zum Thema „Architecture“ angefordert und den Markdown des fertigen Artikels angezeigt.

In [ ]:
article = complete_chain.invoke('Architecture')

Die eigentliche Anzeige des Markdowns wird durch diesen Code übernommen:

In [ ]:
Markdown(article.content)

# **3 | Überblick LangChain Konzepte**
---

Konzepte von LangChain sind grundlegende Bausteine und Prinzipien, die das Framework ausmachen und seine Funktionsweise ermöglichen. Hier sind einige der wichtigsten Konzepte:

+ Chat-Modelle (**Chat models**): LLMs, die über eine Chat-API verfügbar sind und Sequenzen von Nachrichten verarbeiten1.
+ Nachrichten (**Messages**): Kommunikationseinheiten in Chat-Modellen für Ein- und Ausgabe1.
+ Chat-Verlauf (**Chat history**): Eine Sequenz von Nachrichten, die eine Konversation darstellt.
+ Tools (**Tools**): Funktionen mit definierten Schemata für Name, Beschreibung und Argumente.
+ Strukturierte Ausgabe (**Structured output**): Technik, um Chat-Modelle in strukturierten Formaten antworten zu lassen.
+ Retrieval Augmented Generation (**RAG**): Technik zur Verbesserung von Sprachmodellen durch Kombination mit externen Wissensbasen.
+ Prompt-Vorlagen (**Prompt template**s): Komponenten zur Erstellung strukturierter Prompts für LLMs.
+ Chains (**Chains**): Verknüpfungen mehrerer LLMs oder anderer Komponenten für komplexere Anwendungen.
+ Agenten (**Agents**): Nutzen Sprachmodelle, um Aktionssequenzen auszuwählen und mit externen Ressourcen zu interagieren.
+ Retriever (**Retriever**): Komponenten, die relevante Dokumente aus einer Wissensbasis abrufen.

# **4 | Best Practices**




1. **Modularität**:
   - Baue Anwendungen aus kleinen, wiederverwendbaren Komponenten
   - Teste einzelne Komponenten separat
   - Nutze Vererbung für eigene Implementierungen

2. **Fehlerbehandlung**:
   - Implementiere Retry-Mechanismen für API-Aufrufe
   - Validiere Eingaben vor der Verarbeitung
   - Logge wichtige Zwischenschritte

3. **Performance**:
   - Nutze Caching für Embeddings und häufige Anfragen
   - Optimiere Chunk-Größen für deine Anwendung
   - Verwende Batch-Verarbeitung wo möglich

4. **Sicherheit und Kosten**:
   - Setze Limits für Token und Anfragen
   - Überwache API-Kosten
   - Implementiere Rate-Limiting

5. **Testing**:
   - Schreibe Unit-Tests für Chains und Parser
   - Teste mit verschiedenen Eingabeformaten
   - Simuliere API-Aufrufe in Tests



# **5 | Weiterführende Ressourcen**
---



- [Offizielle LangChain Dokumentation](https://python.langchain.com/docs/)
- [LangChain GitHub Repository](https://github.com/langchain-ai/langchain)
- [LangChain Cookbook mit Beispielen](https://github.com/langchain-ai/langchain/tree/master/cookbook)

# **A | Aufgabe**
---

Die Aufgabestellungen unten bieten Anregungen, Sie können aber auch gerne eine andere Herausforderung angehen.


<p><font color='black' size="5">
Erstellen einer LangChain-Kette zur Textanalyse
</font></p>

Entwickeln Sie eine Verarbeitungskette, die einen Text analysiert und verschiedene Informationen darüber extrahiert.

**Schritte:**
1. Erstelle ein ChatPromptTemplate, das ein LLM anweist, einen Text zu analysieren
2. Die Analyse soll folgende Aspekte umfassen:
   - Hauptthema des Textes
   - Tonalität (formal, informell, etc.)
   - Schlüsselwörter (5-10)
   - Kurze Zusammenfassung (max. 3 Sätze)
3. Formatiere die Ausgabe strukturiert mit Markdown-Überschriften
4. Teste die Kette mit mindestens zwei verschiedenen Texten




<p><font color='black' size="5">
Few-Shot-Learning für Textklassifikation
</font></p>

Erstellen Sie ein System, das mithilfe von Few-Shot-Learning Texte in vorgegebene Kategorien klassifiziert.

**Schritte:**
1. Definiere 3-5 Kategorien für die Klassifikation (z.B. Sport, Politik, Technologie, Kultur)
2. Erstelle einen FewShotPromptTemplate mit Beispielen für jede Kategorie
3. Entwickle eine Chain, die neue Texte klassifiziert
4. Implementiere eine Funktion, die neben der Kategorie auch eine Begründung für die Einordnung liefert
5. Teste das System mit verschiedenen Texten, die nicht in den Beispielen vorkommen


<p><font color='black' size="5">
Q&A-System mit Sequential Chain
</font></p>

Entwickeln Sie ein System, das Fragen zu einem gegebenen Kontext beantwortet, aber zuerst den Kontext zusammenfasst und dann die Frage beantwortet.


**Schritte:**

+ Erstellen Sie zwei Templates:
    + Ein Template zur Zusammenfassung des Kontexts
    + Ein Template zur Beantwortung einer Frage basierend auf der Zusammenfassung
+ Verbinden Sie die Templates in einer sequentiellen Kette
+ Implementieren Sie einen Mechanismus, der die Zusammenfassung und die Antwort getrennt zurückgibt
+ Nutzen Sie LCEL (LangChain Expression Language) für die Verkettung
+ Testen Sie das System mit verschiedenen Kontexten und Fragen